Lambda School Data Science

*Unit 4, Sprint 2, Module 4*

---

# Neural Networks & GPUs (Prepare)
*aka Hyperparameter Tuning*

*aka Big Servers for Big Problems*

## Learning Objectives
* <a href="#p1">Part 1</a>: Describe the major hyperparemeters to tune
* <a href="#p2">Part 2</a>: Implement an experiment tracking framework
* <a href="#p3">Part 3</a>: Search the hyperparameter space using RandomSearch (Optional)

In [11]:
wandb_group = "ds8"
wandb_project = "ds13_inclass"

In [2]:
%%capture

!pip install wandb

In [3]:
!wandb login 79680cd73313c44891d9a7ada90cb08599decd6a

wandb: Appending key for api.wandb.ai to your netrc file: /home/tmbern/.netrc
Successfully logged in to Weights & Biases!


# Hyperparameter Options (Learn)
<a id="p1"></a>

## Overview

Hyperparameter tuning is much more important with neural networks than it has been with any other models that we have considered up to this point. Other supervised learning models might have a couple of parameters, but neural networks can have dozens. These can substantially affect the accuracy of our models and although it can be a time consuming process is a necessary step when working with neural networks.
​
Hyperparameter tuning comes with a challenge. How can we compare models specified with different hyperparameters if our model's final error metric can vary somewhat erratically? How do we avoid just getting unlucky and selecting the wrong hyperparameter? This is a problem that to a certain degree we just have to live with as we test and test again. However, we can minimize it somewhat by pairing our experiments with Cross Validation to reduce the variance of our final accuracy values.

### Load Boston Housing Data

In [4]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

### Normalizing Input Data

It's not 100% necessary to normalize/scale your input data before feeding it to a neural network, the network can learn the appropriate weights to deal with data of as long as it is numerically represented,  but it is recommended as it can help **make training faster** and **reduces the chances that gradient descent might get stuck in a local optimum**.

<https://stackoverflow.com/questions/4674623/why-do-we-have-to-normalize-the-input-for-an-artificial-neural-network>

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])

[[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
   0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
  -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
   1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
  -1.30850006]
 [-0.40149354 -0.48361547 -0.86940196 -0.25683275 -0.3615597  -0.3245576
  -1.23667187  1.10717989 -0.51114231 -1.094663    0.78447637  0.44807713
  -0.65292624]
 [-0.0056343  -0.48361547  1.0283258  -0.25683275  1.32861221  0.15364225
   0.69480801 -0.57857203  1.67588577  1.5652875   0.78447637  0.3898823
   0.26349695]
 [-0.37502238 -0.48361547 -0.54747912 -0.25683275 -0.54935658 -0.78865126
   0.18954148  0.48371503 -0.51114231 -0.71552978  0.51145832  0.38669063
  -0.13812828]
 [ 0.58963463 -0.48361547

### Model Validation using an automatic verification Dataset

Instead of doing seperate train test split class, Keras has a really nice feature that you can set the validation.data argument when fitting your model and Keras will take that portion of your test data and use it as a validation dataset. 

In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = x_train.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Fit Model
model.fit(x_train, y_train, 
          validation_data=(x_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Epoch 1/75
41/41 [==============================] - 0s 5ms/step - loss: 535.2932 - mae: 21.1716 - val_loss: 479.6481 - val_mae: 19.8575
Epoch 2/75
41/41 [==============================] - 0s 2ms/step - loss: 334.2502 - mae: 15.7979 - val_loss: 200.4972 - val_mae: 12.0578
Epoch 3/75
41/41 [==============================] - 0s 2ms/step - loss: 108.7122 - mae: 7.8986 - val_loss: 68.8567 - val_mae: 6.3863
Epoch 4/75
41/41 [==============================] - 0s 2ms/step - loss: 45.8006 - mae: 4.8792 - val_loss: 39.8753 - val_mae: 4.9301
Epoch 5/75
41/41 [==============================] - 0s 2ms/step - loss: 29.3793 - mae: 3.8345 - val_loss: 31.0684 - val_mae: 4.4556
Epoch 6/75
41/41 [==============================] - 0s 2ms/step - loss: 23.9873 - mae: 3.4497 - val_loss: 27.2114 - val_mae: 4.1196
Epoch 7/75
41/41 [==============================] - 0s 2ms/step - loss: 20.7134 - mae: 3.1989 - val_loss: 25.6832 - val_mae: 3.9193
Epoch 8/75
41/41 [==============================] - 0s 2ms/step - l

Epoch 63/75
41/41 [==============================] - 0s 2ms/step - loss: 5.7557 - mae: 1.7276 - val_loss: 19.6160 - val_mae: 2.7772
Epoch 64/75
41/41 [==============================] - 0s 2ms/step - loss: 5.8391 - mae: 1.7580 - val_loss: 17.2860 - val_mae: 2.6360
Epoch 65/75
41/41 [==============================] - 0s 2ms/step - loss: 5.8465 - mae: 1.7202 - val_loss: 18.4598 - val_mae: 2.6889
Epoch 66/75
41/41 [==============================] - 0s 3ms/step - loss: 5.7231 - mae: 1.7118 - val_loss: 18.3015 - val_mae: 2.6690
Epoch 67/75
41/41 [==============================] - 0s 2ms/step - loss: 5.7401 - mae: 1.7187 - val_loss: 18.6613 - val_mae: 2.7771
Epoch 68/75
41/41 [==============================] - 0s 2ms/step - loss: 5.8220 - mae: 1.7221 - val_loss: 18.8042 - val_mae: 2.7661
Epoch 69/75
41/41 [==============================] - 0s 3ms/step - loss: 5.5102 - mae: 1.6815 - val_loss: 18.2877 - val_mae: 2.7145
Epoch 70/75
41/41 [==============================] - 0s 2ms/step - loss: 5.6

### Hyperparameter Tuning Approaches:

#### 1) Babysitting AKA "Grad Student Descent".

If you fiddled with any hyperparameters yesterday, this is basically what you did. This approach is 100% manual and is pretty common among researchers where finding that 1 exact specification that jumps your model to a level of accuracy never seen before is the difference between publishing and not publishing a paper. Of course the professors don't do this themselves, that's grunt work. This is also known as the fiddle with hyperparameters until you run out of time method.

#### 2) Grid Search

Grid Search is the Grad Student galaxy brain realization of: why don't I just specify all the experiments I want to run and let the computer try every possible combination of them while I go and grab lunch. This has a specific downside in that if I specify 5 hyperparameters with 5 options each then I've just created 5^5 combinations of hyperparameters to check. Which means that I have to train 3125 different versions of my model Then if I use 5-fold Cross Validation on that then my model has to run 15,525 times. This is the brute-force method of hyperparameter tuning, but it can be very profitable if done wisely. 

When using Grid Search here's what I suggest: don't use it to test combinations of different hyperparameters, only use it to test different specifications of **a single** hyperparameter. It's rare that combinations between different hyperparameters lead to big performance gains. You'll get 90-95% of the way there if you just Grid Search one parameter and take the best result, then retain that best result while you test another, and then retain the best specification from that while you train another. This at least makes the situation much more manageable and leads to pretty good results. 

#### 3) Random Search

Do Grid Search for a couple of hours and you'll say to yourself - "There's got to be a better way." Enter Random Search. For Random search you specify a hyperparameter space and it picks specifications from that randomly, tries them out, gives you the best results and says - That's going to have to be good enough, go home and spend time with your family. 

Grid Search treats every parameter as if it was equally important, but this just isn't the case, some are known to move the needle a lot more than others (we'll talk about that in a minute). Random Search allows searching to be specified along the most important parameter and experiments less along the dimensions of less important hyperparameters. The downside of Random search is that it won't find the absolute best hyperparameters, but it is much less costly to perform than Grid Search. 

#### 4) Bayesian Methods

One thing that can make more manual methods like babysitting and gridsearch effective is that as the experimenter sees results he can then make updates to his future searches taking into account the results of past specifications. If only we could hyperparameter tune our hyperparameter tuning. Well, we kind of can. Enter Bayesian Optimization. Neural Networks are like an optimization problem within an optimization problem, and Bayesian Optimization is a search strategy that tries to take into account the results of past searches in order to improve future ones. Check out the new library `keras-tuner` for easy implementations of Bayesian methods. 


## What Hyperparameters are there to test?

- batch_size
- training epochs
- optimization algorithms
- learning rate
- momentum
- activation functions
- dropout regularization
- number of neurons in the hidden layer

There are more, but these are the most important.

## Follow Along

## Batch Size

Batch size determines how many observations the model is shown before it calculates loss/error and updates the model weights via gradient descent. You're looking for a sweet spot here where you're showing it enough observations that you have enough information to updates the weights, but not such a large batch size that you don't get a lot of weight update iterations performed in a given epoch. Feed-forward Neural Networks aren't as sensitive to bach_size as other networks, but it is still an important hyperparameter to tune. Smaller batch sizes will also take longer to train. 

Traditionally, batch size is set in powers of 2 starting at 32 up to 512. Keras defaults to a batch size of 32 if you do not specify it. Yann LeCun famously Twitted: 

> Training with large minibatches is bad for your health.
More importantly, it's bad for your test error.
Friends dont let friends use minibatches larger than 32.

Check out this paper for more reference on his tweet. https://arxiv.org/abs/1804.07612. 

Check out this SO question on why batch size is typically set in powers of two: https://datascience.stackexchange.com/questions/20179/what-is-the-advantage-of-keeping-batch-size-a-power-of-2



In [7]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8, 16, 32, 64, 128],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.656268572807312 using {'batch_size': 8, 'epochs': 20}
Means: 0.656268572807312, Stdev: 0.02789848407498076 with: {'batch_size': 8, 'epochs': 20}
Means: 0.6263644933700562, Stdev: 0.034835517801393864 with: {'batch_size': 16, 'epochs': 20}
Means: 0.6041846990585327, Stdev: 0.01571390893051874 with: {'batch_size': 32, 'epochs': 20}
Means: 0.6301842093467712, Stdev: 0.0507089659930049 with: {'batch_size': 64, 'epochs': 20}
Means: 0.5533825695514679, Stdev: 0.07720318696104948 with: {'batch_size': 128, 'epochs': 20}


## Optimizer

Remember that there's a different optimizers [optimizers](https://keras.io/optimizers/). At some point, take some time to read up on them a little bit. "adam" usually gives the best results. The thing to know about choosing an optimizer is that different optimizers have different hyperparameters like learning rate, momentum, etc. So based on the optimizer you choose you might also have to tune the learning rate and momentum of those optimizers after that. 

## Learning Rate

Remember that the Learning Rate is a hyperparameter that is specific to your gradient-descent based optimizer selection. A learning rate that is too high will cause divergent behavior, but a Learning Rate that is too low will fail to converge, again, you're looking for the sweet spot. I would start out tuning learning rates by orders of magnitude: [.001, .01, .1, .2, .3, .5] etc. I wouldn't go above .5, but you can try it and see what the behavior is like. 

Once you have narrowed it down, make the window even smaller and try it again. If after running the above specification your model reports that .1 is the best optimizer, then you should probably try things like [.05, .08, .1, .12, .15] to try and narrow it down. 

It can also be good to tune the number of epochs in combination with the learning rate since the number of iterations that you allow the learning rate to reach the minimum can determine if you have let it run long enough to converge to the minimum. 

<img align="left" src="https://miro.medium.com/max/918/0*C5kIkoBwht0fXRgs.jpeg" width=200>
<br></br>
<br></br>

In [8]:
from tensorflow.keras.optimizers import Adam

# Function to create model, required for KerasClassifier
def create_model(lr=0.01):
    # create model
    opt = Adam(learning_rate=lr)
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [8],
              'epochs': [20],
              'lr': [.001, .01, .1, .2]
             }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.718835425376892 using {'batch_size': 8, 'epochs': 20, 'lr': 0.01}
Means: 0.6328240394592285, Stdev: 0.05292509131988432 with: {'batch_size': 8, 'epochs': 20, 'lr': 0.001}
Means: 0.718835425376892, Stdev: 0.059960844866939145 with: {'batch_size': 8, 'epochs': 20, 'lr': 0.01}
Means: 0.6511586427688598, Stdev: 0.05244526932680711 with: {'batch_size': 8, 'epochs': 20, 'lr': 0.1}
Means: 0.6511586427688598, Stdev: 0.05244526932680711 with: {'batch_size': 8, 'epochs': 20, 'lr': 0.2}


## Momentum

Momentum is a hyperparameter that is more commonly associated with Stochastic Gradient Descent. SGD is a common optimizer because it's what people understand and know, but I doubt it will get you the best results, you can try hyperparameter tuning its attributes and see if you can beat the performance from adam. Momentum is a property that decides the willingness of an optimizer to overshoot the minimum. Imagine a ball rolling down one side of a bowl and then up the opposite side a little bit before settling back to the bottom. The purpose of momentum is to try and escale local minima.

## Activation Functions

We've talked about this a little bit, typically you'l want to use ReLU for hidden layers and either Sigmoid, or Softmax for output layers of binary and multi-class classification implementations respectively, but try other activation functions and see if you can get any better results with sigmoid or tanh or something. There are a lot of activation functions that we haven't really talked about. Maybe you'll get good results with them. Maybe you won't. :) <https://keras.io/activations/>

## Network Weight Initialization

You saw how big of an effect the way that we initialize our network's weights can have on our results. There are **a lot** of what are called initialization modes. I don't understand all of them, but they can have a big affect on your model's initial accuracy. Your model will get further with less epochs if you initialize it with weights that are well suited to the problem you're trying to solve.

`init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']`

## Dropout Regularization and the Weight Constraint

the Dropout Regularization value is a percentage of neurons that you want to be randomly deactivated during training. The weight constraint is a second regularization parameter that works in tandem with dropout regularization. You should tune these two values at the same time. 

Using dropout on visible vs hidden layers might have a different effect. Using dropout on hidden layers might not have any effect while using dropout on hidden layers might have a substantial effect. You don't necessarily need to turn use dropout unless you see that your model has overfitting and generalizability problems.

## Neurons in Hidden Layer 

Remember that when we only had a single perceptron our model was only able to fit to linearly separable data, but as we have added layers and nodes to those layers our network has become a powerhouse of fitting nonlinearity in data. The larger the network and the more nodes generally the stronger the network's capacity to fit nonlinear patterns in data. The more nodes and layers the longer it will take to train a network, and higher the probability of overfitting. The larger your network gets the more you'll need dropout regularization or other regularization techniques to keep it in check. 

Typically depth (more layers) is more important than width (more nodes) for neural networks. This is part of why Deep Learning is so highly touted. Certain deep learning architectures have truly been huge breakthroughs for certain machine learning tasks. 

You might borrow ideas from other network architectures. For example if I was doing image recognition and I wasn't taking cues from state of the art architectures like resnet, alexnet, googlenet, etc. Then I'm probably going to have to do a lot more experimentation on my own before I find something that works.

There are some heuristics, but I am highly skeptical of them. I think you're better off experimenting on your own and forming your own intuition for these kinds of problems. 

- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

## Challenge
You will be expected to tune several hyperparameters in today's module project. 

# Experiment Tracking Framework (Learn)
<a id="p2"></a>

## Overview

You will notice quickly that managing the results of all the experiments you are running becomes challenging. Which set of parameters did the best? Are my results today different than my results yesterday? Although we use Ipython Notebooks to work, the format is not well suited to logging experimental results. Enter experiment tracking frameworks like [Comet.ml](https://comet.ml) and [Weights and Biases](https://wandb.ai/).

Those tools will help you track your experiments, store the results, and the code associated with those experiments. Experimental results can also be readily visualized to see changes in performance across any metric you care about. Data is sent to the tool as each epoch is completed, so you can also see if your model is converging. Let's check out Weights & Biases today. 

## Follow Along

Make sure you login into `wandb` in the terminal before running the next cell. 

In [13]:
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.layers import Dropout

In [12]:
wandb.init(project=wandb_project, entity=wandb_group) #Initializes and Experiment; this should be in the same cell as the fit statement

# Important Hyperparameters
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 32

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

wandb: Wandb version 0.8.36 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/50
9/9 [==============================] - 0s 16ms/step - loss: 526.3895 - mse: 526.3895 - mae: 21.2134 - val_loss: 584.1267 - val_mse: 584.1267 - val_mae: 22.0330
Epoch 2/50
9/9 [==============================] - 0s 9ms/step - loss: 468.2961 - mse: 468.2961 - mae: 19.7768 - val_loss: 511.8218 - val_mse: 511.8218 - val_mae: 20.2564
Epoch 3/50
9/9 [==============================] - 0s 8ms/step - loss: 389.4769 - mse: 389.4769 - mae: 17.5525 - val_loss: 404.6097 - val_mse: 404.6097 - val_mae: 17.3682
Epoch 4/50
9/9 [==============================] - 0s 10ms/step - loss: 278.5723 - mse: 278.5723 - mae: 14.1660 - val_loss: 270.8217 - val_mse: 270.8217 - val_mae: 13.4384
Epoch 5/50
9/9 [==============================] - 0s 10ms/step - loss: 160.9369 - mse: 160.9369 - mae: 10.1819 - val_loss: 156.6449 - val_mse: 156.6449 - val_mae: 9.5459
Epoch 6/50
9/9 [==============================] - 0s 8ms/step - loss: 90.6565 - mse: 90.6565 - mae: 7.3322 - val_loss: 112.7574 - val_mse: 112.7574 

### Your Turn

Pick a few hyparameters that we *have not* tuned. Using the same code above, try changing a few parameters you're interested in and submitting the results to weights & biases. :) 

In [16]:
wandb.init(project=wandb_project, entity=wandb_group) #Initializes and Experiment

wandb.config.epochs = 50
wandb.config.batch_size = 32


# Create Model
model = Sequential()
model.add(Dense(120, activation='relu', input_shape=(inputs,)))
model.add(Dropout(0.2))
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(120, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])



# Fit Model
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

wandb: Wandb version 0.8.36 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/50
9/9 [==============================] - 0s 16ms/step - loss: 514.8749 - mse: 514.8749 - mae: 20.8067 - val_loss: 532.4934 - val_mse: 532.4934 - val_mae: 20.6799
Epoch 2/50
9/9 [==============================] - 0s 9ms/step - loss: 387.5506 - mse: 387.5506 - mae: 17.2475 - val_loss: 358.6980 - val_mse: 358.6980 - val_mae: 16.2500
Epoch 3/50
9/9 [==============================] - 0s 9ms/step - loss: 222.0815 - mse: 222.0815 - mae: 12.5811 - val_loss: 170.2490 - val_mse: 170.2490 - val_mae: 10.4169
Epoch 4/50
9/9 [==============================] - 0s 9ms/step - loss: 100.8014 - mse: 100.8014 - mae: 8.2463 - val_loss: 103.1693 - val_mse: 103.1693 - val_mae: 8.2981
Epoch 5/50
9/9 [==============================] - 0s 9ms/step - loss: 81.6818 - mse: 81.6818 - mae: 6.8598 - val_loss: 77.4397 - val_mse: 77.4397 - val_mae: 6.6907
Epoch 6/50
9/9 [==============================] - 0s 9ms/step - loss: 47.7846 - mse: 47.7846 - mae: 5.1534 - val_loss: 64.2234 - val_mse: 64.2234 - val_mae: 

9/9 [==============================] - 0s 6ms/step - loss: 13.3638 - mse: 13.3638 - mae: 2.6670 - val_loss: 16.8473 - val_mse: 16.8473 - val_mae: 2.7971


## Challenge

You will be expected to use Weights & Biases to try to tune your model during your module assignment today. 

# Hyperparameters with RandomSearchCV (Learn)

## Overview

Basically `GridSearchCV` takes forever. You'll want to adopt a slightly more sophiscated strategy.

Let's also take a look at an alternative with Keras-Tuner.

In [18]:
%%capture
!pip install keras-tuner

## Follow Along

In [23]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

"""
This model Tunes:
- Number of Neurons in the Hidden Layer
- Learning Rate in Adam

"""

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='mse',
        metrics=['mae'])
    
    return model

In [26]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='./keras-tuner-trial',
    project_name='helloworld')

In [27]:
tuner.search_space_summary()

In [28]:
tuner.search(X, y,
             epochs=5,
             validation_split=.15)

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11/11 [==============================] - ETA: 0s - loss: 410.7359 - mae: 19.28 - 0s 10ms/step - loss: 550.3939 - mae: 21.6424 - val_loss: 618.6282 - val_mae: 22.9911
Epoch 2/5
11/11 [==============================] - ETA: 0s - loss: 665.1523 - mae: 22.99 - 0s 5ms/step - loss: 504.0105 - mae: 20.6051 - val_loss: 568.2460 - val_mae: 21.8909
Epoch 3/5
11/11 [==============================] - ETA: 0s - loss: 468.7941 - mae: 19.77 - 0s 4ms/step - loss: 456.4669 - mae: 19.4752 - val_loss: 513.4958 -

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11/11 [==============================] - ETA: 0s - loss: 659.2016 - mae: 22.90 - 0s 9ms/step - loss: 574.0175 - mae: 22.1243 - val_loss: 660.8386 - val_mae: 23.8068
Epoch 2/5
11/11 [==============================] - ETA: 0s - loss: 593.4406 - mae: 22.02 - 0s 4ms/step - loss: 547.1961 - mae: 21.5561 - val_loss: 631.0400 - val_mae: 23.2177
Epoch 3/5
11/11 [==============================] - ETA: 0s - loss: 642.8376 - mae: 22.75 - 0s 4ms/step - loss: 520.9777 - mae: 20.9760 - val_loss: 601.9401 - 

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11/11 [==============================] - ETA: 0s - loss: 561.3997 - mae: 21.45 - 0s 9ms/step - loss: 447.1981 - mae: 19.0004 - val_loss: 382.3875 - val_mae: 17.0809
Epoch 2/5
11/11 [==============================] - ETA: 0s - loss: 364.1661 - mae: 16.21 - 0s 4ms/step - loss: 210.1820 - mae: 12.1533 - val_loss: 142.8671 - val_mae: 9.3346
Epoch 3/5
11/11 [==============================] - ETA: 0s - loss: 139.9738 - mae: 8.354 - 0s 4ms/step - loss: 81.1360 - mae: 7.0348 - val_loss: 80.9142 - val_

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11/11 [==============================] - ETA: 0s - loss: 522.2888 - mae: 21.68 - 0s 9ms/step - loss: 576.5069 - mae: 22.2052 - val_loss: 671.0129 - val_mae: 24.0689
Epoch 2/5
11/11 [==============================] - ETA: 0s - loss: 676.4475 - mae: 24.13 - 0s 4ms/step - loss: 573.8361 - mae: 22.1474 - val_loss: 668.1917 - val_mae: 24.0117
Epoch 3/5
11/11 [==============================] - ETA: 0s - loss: 684.0388 - mae: 23.99 - 0s 4ms/step - loss: 571.2206 - mae: 22.0909 - val_loss: 665.3286 - 

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11/11 [==============================] - ETA: 0s - loss: 661.9510 - mae: 23.99 - 0s 9ms/step - loss: 571.6116 - mae: 22.2188 - val_loss: 655.1550 - val_mae: 23.8188
Epoch 2/5
11/11 [==============================] - ETA: 0s - loss: 626.8175 - mae: 23.52 - 0s 5ms/step - loss: 554.0580 - mae: 21.8246 - val_loss: 636.8917 - val_mae: 23.4281
Epoch 3/5
11/11 [==============================] - ETA: 0s - loss: 626.0076 - mae: 22.82 - 0s 4ms/step - loss: 537.1421 - mae: 21.4391 - val_loss: 618.5886 - 

INFO:tensorflow:Oracle triggered exit


In [29]:
tuner.results_summary()

## Challenge

Try to apply RandomSearchCV to your module project today. 

# Review
* <a href="#p1">Part 1</a>: Describe the major hyperparemeters to tune
    - Activation Functions
    - Optimizer
    - Number of Layers
    - Number of Neurons
    - Batch Size
    - Dropout Regulaization
    - Learning Rate
    - Number of Epochs
    - and many more
* <a href="#p2">Part 2</a>: Implement an experiment tracking framework
    - Weights & Biases
    - Comet.ml
    - By Hand / GridSearch
* <a href="#p3">Part 3</a>: Search the hyperparameter space using RandomSearch
    - Sklearn still useful (haha)
    - Integration with Wieghts & Biases
* <a href="#p4">Part 4</a>: Discuss emerging hyperparameter tuning strategies
    - Bayesian Optimization
    - Hyperopt
    - Genetic Evolution

# Sources

## Additional Reading
- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
- https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/
- https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
- https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/
- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/